# OpenStreetMap Sample Project - Data Wrangling with MongoDB

Anita Weck (Map Area: District Bad Kreuznach - Rhinelande Palatinate - Germany

https://www.openstreetmap.org/#map=12/49.8241/7.7436

## Problems encountered in the map

After downloading the open street map area of the district of Bad Kreuznach, shaping and importing it in MongoDB I realized the following problems:
- city names are doubled because of misspelled spelling letters (wrong 'hein' vs. correct 'heim'), different case styles (small letters vs. case-sensitive style), different style in using white spaces before and after a hyphen (wrong 'Bad Sobernheim-Steinhardt' vs. 'Bad Sobernheim - Steinhard; here also wrong naming of city district)
- inconsistent street names (wrong 'strasse' vs. correct 'straße) in german style of street naming
- german characters are not encoded correctly

### City names

Firstly the data are audited after reshaping the tags for MongoDB. By using the following aggregrate function the problems for city names above are encountered.

In [ ]:
> db.P3_Meddersheim_Germany.aggregate([{"$match":{"address.city":{"$exists":1}}},{"$group":{"_id":"$address.city"}},{"$sort":{"_id":1}}])

The problems look like this in the output:

<br>...</br>
<br>{ "_id" : "Bad Kreuznach" }</br>
<br>{ "_id" : "Bad Kreuznach - Rüdesheim" }</br>
<br>{ "_id" : "Bad Münster" }</br>
<br>{ "_id" : "Bad Münster am Stei<ins>n-E</ins>bernburg" }</br>
<br>{ "_id" : "Bad Sobernheim" }</br>
<br>{ "_id" : "Bad Sobernheim<ins> - </ins>Steinhard" }</br>
<br>{ "_id" : "Bad Sobernhei<ins>m-S</ins>teinhardt" }</br>
<br>{ "_id" : "Bad Sobernhei<ins>n</ins>" }</br>
<br>...</br>
<br>{ "_id" : "<ins>b</ins>ad Kreuznach" }</br>

The result after cleaning looks like this in the database after the new json-file is red in.

<br>...</br>
<br>{ "_id" : "Bad Kreuznach" }</br>
<br>{ "_id" : "Bad Kreuznach - Rüdesheim" }</br>
<br>{ "_id" : "Bad Münster" }</br>
<br>{ "_id" : "Bad Münster am Stein - Ebernburg" }</br>
<br>{ "_id" : "Bad Sobernheim" }</br>
<br>{ "_id" : "Bad Sobernheim - Steinhard" }</br>
<br>{ "_id" : "Bad Sobernheim - Steinhardt" }</br>
<br>...</br>

The chosen way of cleaning these city names is done via defining a pattern and checking it against a regular expression.
Therefor that very often the string 'hein' appears in this example osm-dataset (and in most cases is correct) a check for the this pattern at the end of string was made.
The string 'bad' starts with a small letter, but should start with a capital letter (in this dataset 'bad' does not appear in another meaning that's why a check for only the beginning of a string with a whitespace was renounced).
The last problem which was solved was the inconsistency in 

In [ ]:
#pattern definitions for wrangling address contents
p_wrangle = re.compile("(?=hein$)", re.IGNORECASE)
p_bad = re.compile("bad")
p_line = re.compile("\w-\w")
# correct city names spelling mistakes, sensitive style and consistent hyphen whitespacing
if child_name[0] == "city":
    if re.search(p_wrangle,child.get('v')) != None:
        child_content = child.get('v')
        child_content_new = child_content.replace("hein", "heim")
    elif re.search(p_bad,child.get('v')) != None:
        child_content = child.get('v')
        child_content_new = child_content.replace("bad", "Bad")
    elif re.search(p_line, child.get('v')) != None:
        child_content = child.get('v')
        child_content_new = child_content.replace("-"," - ") 
    if child_content_new != None:
        add_address["city"] = child_content_new

### Street names

By auditing the database for a typical german mistake in street naming by the following aggregate function, using "$regex" for finding a string,

In [ ]:
> db.P3_Meddersheim_Germany.aggregate([{"$match":{"address.street":{"$regex":"strasse"}}},{"$group":{"_id":"$address.street","count":{"$sum":1}}},{"$sort":{"_id":1}}])

shows that typical inconsistency:

{ "_id" : "Winzerstra<ins>ss</ins>e", "count" : 1 }

Versus the other correct street names:

In [ ]:
> db.P3_Meddersheim_Germany_2.aggregate([{"$match":{"address.street":{"$regex":"straße"}}},{"$group":{"_id":"$address.street","count":{"$sum":1}}},{"$sort":{"_id":1}}])

<br>{ "_id" : "Altmühlenstraße", "count" : 18 }</br>
<br>{ "_id" : "Bachstraße", "count" : 18 }</br>
<br>...</br>

The problem was solved via a consistent exchanging of the wrong string (pattern definition) with the correct string (containing the german letter 'ß') in the following way. A check was done with the aggregate function again, which does not bring back a result for the expression {"$regex":"strasse"}.

In [ ]:
#pattern definitions for wrangling address contents
p_street = re.compile("strasse")
# conistent street naming, use always german characters
if child_name[0] == "street":
    if re.search(p_street,child.get('v')) != None:
        print child_name, child.get('v')
        child_content = child.get('v')
        child_content_new = child_content.replace("strasse", "straße")
        print p_wrangle, child_content_new
        add_address["street"] = child_content_new

This special character problem is solved with the coding definition explained in the following chapter.

### Encoding for german letters 

To ensure the correct encoding of german letters (like 'ß') and to avoid and exception when executing the python-code

In [ ]:
child_content_new = child_content.replace("strasse", "straße")

the coding definition at the beginning of the py-file is necessary to ensure correct encoding for the json-object-file and compiling of the code.

In [ ]:
# -*- coding: utf-8 -*-

### No problems are found in the postal codes

The postcodes are correct and consistent at least for the included cities in the chosen map. This was checked with the following function.

In [ ]:
> db.P3_Meddersheim_Germany_2.aggregate([{"$match":{"address.postcode":{"$exists":1}}},{"$group":{"_id":"$address.postcode","city":{"$addToSet":"$address.city"}}},{"$sort":{"address.postcode":1}}])

<br>{ "_id" : "55450", "city" : [ "Bad Kreuznach - Rüdesheim", "Langenlonsheim" ] }</br>
<br>{ "_id" : "55559", "city" : [ "Altenbamberg", "Bretzenheim" ] }</br>
<br>{ "_id" : "55568", "city" : [ "Bad Kreuznach", "Bad Sobernheim", "Staudernheim" ] }</br>
<br>{ "_id" : "67826", "city" : [ "Bad Kreuznach", "Hallgarten" ] }</br>
<br>{ "_id" : "55571", "city" : [ "Bad Sobernheim", "Odernheim" ] }</br>
<br>{ "_id" : "55595", "city" : [ "Allenfeld", "Bad Kreuznach - Rüdesheim", "Mandel", "Hargesheim", "Winterbach", "Spall", "Weinsheim", "Argenschwang", "Sommerloch", "Boos", "Bad Kreuznach", "Roxheim", "Bad Sobernheim", "Traisen" ] }</br>
<br>{ "_id" : "55585", "city" : [ "Bad Kreuznach", "Altenbamberg", "Niederhausen" ] }</br>
<br>{ "_id" : "55543", "city" : [ "Hargesheim", "Spall", "Rüdesheim", "Bad Kreuznach", "Bad Münster", "Bad Sobernheim", "Traisen", "Bad Sobernheim - Steinhard" ] }</br>
<br>{ "_id" : "55545", "city" : [ "Winzenheim", "Rüdesheim", "Hargesheim", "Bad Sobernheim - Steinhard", "Bad Kreuznach" ] }</br>
<br>{ "_id" : "55569", "city" : [ "Auen", "Monzingen" ] }</br>
<br>{ "_id" : "55596", "city" : [ "Schloßböckelheim - Kolonie", "Oberstreit", "Waldböckelheim" ] }</br>
<br>{ "_id" : "55546", "city" : [ "Fürfeld" ] }</br>
<br>{ "_id" : "55452", "city" : [ "Norheim", "Guldental" ] }</br>
<br>{ "_id" : "55593", "city" : [ "Staudernheim", "Rüdesheim" ] }</br>
<br>{ "_id" : "67824", "city" : [ "Feilbingert" ] }</br>
<br>{ "_id" : "55566", "city" : [ "Bad Sobernheim - Steinhardt", "Ippenschied", "Meddersheim", "Waldböckelheim", "Bad Sobernheim - Steinhard", "Daubach", "Bad Sobernheim", "Schloßböckelheim - Kolonie", "Bad Kreuznach" ] }</br>
<br>{ "_id" : "55583", "city" : [ "Bad Sobernheim", "Bad Münster", "Bad Kreuznach - Rüdesheim", "Bad Münster am Stein - Ebernburg" ] }</br>

In [ ]:
eim_Germany.aggregate([{"$match":{"address.postcode":{"$exists":1}}},{"$group":{"_id":"$address.postcode","count":{"$sum":1}}},{"$sort":{"count":1}}])

### Shaping of json data elements

Separate the 'addr:'-tag by splitting at the separator colon. The method is called from 'def shape_element(element):'

In [ ]:
#separate definition in case 'addr:' is available in different data types
def address_element(child_name, child):
    child_name = child_name.replace("addr:","")
    child_name = child_name.split(":")
    if len(child_name)<2:
        add_address[child_name[0]] = child.get('v')
    return add_address

Define needed arrays and dictionary for further re-shaping, splitting and organizing the original xml-data and preparing for writing in json-format. At last get type of tag. I tried this type of coding to try a two different ways of re-shaping data. This is only for 'addr:'-tags. Below I used the separator 'colon'for shaping the 'tag'-tags. Main idea was to separat the 'contact:'-data in an own dictionary.

In [ ]:
def shape_element(element):
    node = {}
    openGeoDB = {}
    auto_update = []
    is_in = []
    position = []
    node_refs = []
    relation_member_attrib = []
    relation_member = []
    add_created = {}
    tag_sep = []
    tag_sep_dict = {}
    relation_tag_attrib = {}
    child_name = None
    #set type of parsed data based on tag
    if element.tag =="bounds" or element.tag == "note" or element.tag == "meta" or element.tag == "osm" or\
    element.tag == "node" or element.tag == "way" or element.tag == "relation":
        node["type"] = element.tag

Just to complete get information from tags in xml-file which only appear once.

In [ ]:
# get information from tags which only appear once
    if element.tag =="bounds" or element.tag == "note" or element.tag == "meta" or element.tag == "osm":
        if element.tag == "note":
            node[element.tag] = element.text
        else:
            node[element.tag] = element.attrib
            #print node

Ensure to get all 'openGeoDB' tags, even those with small letters. Prepare for later separation of this tag and saving in own dictionary 'openGeoDB'.

In [ ]:
#shape data
    for child in element:
        child_name = child.get('k')
        p = re.compile('openGeoDB:', re.IGNORECASE)     

Prepare all data within 'is_in' within an array. Split at separator comma. There is also another 'is_in' field within 'openGeoDB'. Put also an array there, but also export this field to json-format.

In [ ]:
#put data in an array, because it is a list of descriptive location
        if child_name == "is_in":
            is_in = child.get('v').split(',')
            node[child_name]=is_in       

The xml-data from 'auto_update' are stored in the array 'auto_update' and splitted by the separator comma. Further the same is done for 'is_in' and both information are stored within the dictionary 'openGeoDB'.
The position data from all tags are taken from the longitude and latitude information and put together in the array 'position'. Also stored in the dictionary 'openGeoDB' within the dictionary 'node'.

In [ ]:
# get all data which contain 'openGeoDB' also in small letters
        elif child_name != None and re.search(p,child_name) != None:
            child_name = p.sub("", child_name)
            if child_name == 'auto_update':
                auto_update = child.get('v').split(',')
                openGeoDB[child_name]=auto_update
            elif child_name == "is_in":
                is_in = child.get('v').split(',')
                openGeoDB[child_name]=is_in
            elif child_name == 'lat':
                position.append(float(child.get('v')))
            elif child_name == 'lon':
                position.append(float(child.get('v')))
            else:
                openGeoDB[child_name] = child.get('v')
            # put latitude and longitude data in an array which are inside 'openGeoDB' data
            openGeoDB['position'] = position
            node['openGeoDB'] = openGeoDB
            #print node

Put all reference information from separate xml-tags together in one array 'node_refs'.

In [ ]:
#check all data which contain a 'ref'-tag
        elif child.get('ref') != None:
                node_refs.append(child.get('ref'))
                node['node_refs'] = node_refs       

Here the 'def address-element(child_name, child)' is called to re-shape the tag 'addr:'. Furthermore separate the other other information by the separator colon and use value before first separator and value after last separator for defining information in json-format.
Otherwise use tag with 'k'(key)-information and 'v'(value)-information for creating key-value pairs in key-value-dictionary whithin 'tag' dictionary. Keep 'tag' as separate dictionary because at least 'type' is also available as key in node-dictionary.
Main idead was to separate the 'contact:'-data in an own dictionary to reach easier email and phone information from contact dictionary.

In [ ]:
#shape all data in 'tag'-tag because some names are the same like in nodes => type, so cluster 'tag' is kept
        elif child.tag == "tag":
            child_name = child.get('k')
            #shape address data in dictionary address
            if child_name != None and child.get('k').startswith("addr:"):
                add_address = address_element(child_name, child)
                node["address"]=add_address
            #shape all other data which use ':' as a separator depending on amount of this separator
            elif re.search(':', child_name) != None:
                tag_sep = child_name.split(':')
                i.append(len(tag_sep))
                if len(tag_sep) == 2:
                    tag_sep_dict[tag_sep[1]] = child.get('v') 
                    node[tag_sep[0]] = tag_sep_dict
                elif len(tag_sep) == 3:
                    tag_sep_dict[tag_sep[2]] = child.get('v') 
                    node[tag_sep[0]] = tag_sep_dict
                elif len(tag_sep) == 4:
                    tag_sep_dict[tag_sep[3]] = child.get('v') 
                    node[tag_sep[0]] = tag_sep_dict   
            else:
                relation_tag_attrib[child.get('k')] = child.get('v')
                node['tag'] = relation_tag_attrib         

Get member data and shape for json-format.

In [ ]:
#shape 'member'-tag data in dicitionary 'member' inside dictionary node
        elif child.tag == "member":
                #print child.get('type')
                relation_member_attrib = child.get('type'), child.get('ref'), child.get('role')
                relation_member.append(relation_member_attrib)
                node['member'] = relation_member
        else:
            if child_name != None:
                node[child_name]=child.get('v')

Put header-information together in a dictionary 'created'. Like done in lesson 6 - Case Study.

In [ ]:
#shape tag header-data in 'CREATED' dictionary, see lesson 6 - Case Study - OpenStreetMap data
    for elem in element.attrib:
        for entry in CREATED:
            if elem == entry and elem != None:
                if element.get(entry) != None:
                    add_created[elem] = element.get(entry)
            elif elem == "id" and elem != None:
                if element.get(elem) != None:
                    node[elem] = element.get(elem) 

Shape longitude and latitude data in node dictionary in an array 'position'. These data define the position of each node.

In [ ]:
#shape position data in positin array for each data-element
    if element.get != None and element.get('lat') or element.get('lon'):
        position = []
        position.append(float(element.get('lat')))   
        position.append(float(element.get('lon')))
        node["pos"]= position
    if len(add_created) > 1:
        node["created"] = add_created
    return node

## Statistical Overview

This section contains basic statistics about the dataset and the MongoDB queries used to gather them.

#### Collection size 

In [ ]:
> db.P3_Meddersheim_Germany_2.dataSize()

61912037 bytes

#### Number of documents in the database 'DW_P3_OSM_after':

In [ ]:
> db.P3_Meddersheim_Germany_2.find().count()

255247

#### Number of the different types (nodes, ways, relations, osm, meta, bounds, note) in descending order sorted:

In [ ]:
> db.P3_Meddersheim_Germany_2.aggregate([{"$group":{"_id":"$type","count":{"$sum":1}}},{"$sort":{"count":-1}}])

<br>{ "_id" : "node", "count" : 218983 }</br>
<br>{ "_id" : "way", "count" : 35578 }</br>
<br>{ "_id" : "relation", "count" : 682 }</br>
<br>{ "_id" : "osm", "count" : 1 }</br>
<br>{ "_id" : "meta", "count" : 1 }</br>
<br>{ "_id" : "bounds", "count" : 1 }</br>
<br>{ "_id" : "note", "count" : 1 }</br>

#### Number of unique users are:

In [ ]:
> db.P3_Meddersheim_Germany_2.distinct("created.uid").length

484

#### Top 3 contributing users are:

In [ ]:
> db.P3_Meddersheim_Germany_2.aggregate([{"$group":{"_id":"$created.user","count":{"$sum":1}}},{"$sort":{"count":-1}},{"$limit":3}])

<br>{ "_id" : "poppei82", "count" : 65353 }</br>
<br>{ "_id" : "paelzer", "count" : 40887 }</br>
<br>{ "_id" : "chrisber", "count" : 31151 }</br>

#### Additional queries 

4. Call query for getting wineries from OpenStreetMap-data
5. Number of chosen types of nodes

In [3]:
import pprint
from pymongo import MongoClient

def get_db():    
    client = MongoClient('localhost',27017)
    db = client['test']
    coll = db.P3_Meddersheim_Germany
    return db

#getting wineries 
def range_query():
    query = {"tag.shop":"winery"}
    return query
    
#getting types of nodes  
def make_pipeline():
    pipeline = []
    #count types of nodes and get them
    pipeline = [{"$match":{"type":"node","type":{"$ne":None},"tag.amenity":{"$ne":None}}},
                {"$group":{"_id":"$tag.amenity","count":{"$sum":1}}},
                {"$sort":{"count":-1}}]
    #alternative pipeline for checking which type is the right one
    #pipeline = [{"$match":{"openGeoDB.type":{"$ne":None}}},
    #            {"$group":{"_id":"$openGeoDB.type","count":{"$sum":1}}},
    #            {"$sort":{"count":-1}}]
    #print pipeline
    return pipeline

def osm_sources(db, pipeline):
    return [doc for doc in db.P3_Meddersheim_Germany.aggregate(pipeline)]

if __name__ == '__main__':
    #read_file(street_map_file)
    db = get_db()
    pipeline = make_pipeline()
    query = range_query()
    result_id = db.P3_Meddersheim_Germany.find(query)
    result = osm_sources(db, pipeline)
    for r in result_id:
        pprint.pprint(r)
    pprint.pprint(result)

{u'_id': ObjectId('5686ecf1cf4e09e2ffdb0808'),
 u'address': {u'city': u'Monzingen',
              u'country': u'DE',
              u'housenumber': u'23',
              u'postcode': u'55569',
              u'street': u'Soonwaldstra\xdfe'},
 u'contact': {u'fax': u'+49 6751 94705', u'phone': u'+49 6751 3847'},
 u'created': {u'changeset': u'13478987',
              u'timestamp': u'2012-10-13T13:23:36Z',
              u'uid': u'39424',
              u'user': u'atressel',
              u'version': u'7'},
 u'id': u'385151392',
 u'pos': [49.8019191, 7.5910752],
 u'tag': {u'landuse': u'farmyard',
          u'name': u'Weingut Jaeger',
          u'shop': u'winery',
          u'tourism': u'wine_cellar'},
 u'type': u'node'}
{u'_id': ObjectId('5686ecf1cf4e09e2ffdb2fa2'),
 u'created': {u'changeset': u'4214878',
              u'timestamp': u'2010-03-23T18:51:38Z',
              u'uid': u'234214',
              u'user': u'Anubis85',
              u'version': u'1'},
 u'id': u'672850551',
 u'pos': [49.82

# Other ideas about the datasets

The field 'is_in' is available twice in the dictionary node and seems to contain the same data. The second time it is stored in node => openGeoDB => is_in dictionary and could be checked if those data are really the same compared to what is directly stored in node => is_in.

The contact data could be grouped according to the 'amenity or shop'-type and contacted for services (e. g. winery services and equipment, vineyard or restaurants support, holiday and tourism service).

Further spelling mistakes can be resolved (e. g. { "_id" : "Bad Sobernheim - Steinhard" }).

The population is described in two elements within 'node': "tag.population" and "openGeoDB.population". This redundant information can be reduced to a single source of information. For checking the size of population and where the most of the people live in this region the data should be available as integer values and not strings.

Additionally the postal codes are available in in the 'openGeoDB.postal_codes' section beside 'address.postcode'. This could be reduced to one field or a pattern might be identified for matching the area code with the zip code. Also an alignment of zip codes with area codes for increasing the efficiency in e. g. glass fibre installation or digitalisation of the postal service via localization with a uniform id usage might be worth for research.

In [ ]:
> db.P3_Meddersheim_Germany_2.aggregate([{"$match":{"openGeoDB.postal_codes":{"$exists":1}}},{"$project":{"_id":0,"openGeoDB.postal_codes":1,"openGeoDB.telephone_area_code":1}},{"$group":{"_id":"$openGeoDB.telephone_area_code","postcode":{"$addToSet":"$openGeoDB.postal_codes"},"count":{"$sum":1}}},{"$sort":{"count":1}}])

<br>{ "_id" : "06754", "postcode" : [ "55569" ], "count" : 1 }</br>
<br>{ "_id" : "06755", "postcode" : [ "55571" ], "count" : 1 }</br>
<br>{ "_id" : "06362", "postcode" : [ "67826" ], "count" : 1 }</br>
<br>{ "_id" : "06756", "postcode" : [ "55595" ], "count" : 2 }</br>
<br>{ "_id" : "06758", "postcode" : [ "55596" ], "count" : 5 }</br>
<br>{ "_id" : "06708", "postcode" : [ "55583", "55585", "55595" ], "count" : 6 }</br>
<br>{ "_id" : "06751", "postcode" : [ "55568", "55569", "55595", "55566" ], "count" : 7 }</br>
<br>{ "_id" : "0671", "postcode" : [ "55546", "55593", "55595", "55545,55543" ], "count" : 16 }</br>

## Additional data exploration using MongoDB queries 

Localization of locations with highest population - the source of these data might be questionable for checking correctness and usage as a reliable source for concentration of population (e. g. checking amount of people living per qm).

In [ ]:
> db.P3_Meddersheim_Germany_2.aggregate([{"$match":{"openGeoDB.name":{"$exists":1}}},{"$project":{"_id":0,"openGeoDB.name":1,"openGeoDB.population":1}},{"$sort":{"population":-1}}])

<br>...</br>
<br>{ "openGeoDB" : { "name" : "Hargesheim", "population" : "2890" } }</br>
<br>{ "openGeoDB" : { "name" : "Braunweiler", "population" : "616" } }</br>
<br>...</br>

Available recycling types in the region e. g. for identification what is missing in this region and why

In [ ]:
> db.P3_Meddersheim_Germany_2.aggregate([{"$match":{"tag.recycling_type":{"$exists":1}}},{"$group":{"_id":"$tag.recycling_type","count":{"$sum":1}}},{"$sort":{"count":-1}}])

<br>{ "_id" : "container", "count" : 32 }</br>
<br>{ "_id" : "centre", "count" : 2 }</br>

Available recycling groups - checking for regrouping and types of recycle stuff. Additional data like frequency of bringing stuff to these places supports the frequency of emptying the containers and transport the recycle things to the recycle centres. The scheduling for the waste management can be optimized and e. g. set in relation to population close to recycling places. 

In [ ]:
> db.P3_Meddersheim_Germany_2.aggregate([{"$match":{"tag.recycling_type":{"$exists":1}}},{"$group":{"_id":"$tag.recycling_type","type":{"$addToSet":"$recycling"},"count":{"$sum":1}}},{"$sort":{"count":-1}}])

{ "_id" : "container", "type" : [ { "green_waste" : "no", "cartons" : "no", "small_appliances" : "no", "plastic_bottles" : "no", "wood" : "no", "plastic_packaging" : "no", "scrap_metal" : "no", "glass_bottles" : "no", "plastic" : "no", "paper" : "no", "glass" : "no", "batteries" : "no", "books" : "no", "magazines" : "no", "cans" : "no", "newspaper" : "no", "waste" : "no", "cardboard" : "no", "electrical_appliances" : "no", "paper_packaging" : "no", "clothes" : "yes" }, { "green_waste" : "no", "cartons" : "no", "small_appliances" : "no", "plastic_bottles" : "no", "wood" : "no", "scrap_metal" : "no", "glass_bottles" : "yes", "plastic" : "no", "paper" : "no", "glass" : "yes", "batteries" : "no", "books" : "no", "magazines" : "no", "cans" : "no", "newspaper" : "no", "waste" : "no", "cardboard" : "no", "electrical_appliances" : "no", "paper_packaging" : "no", "plastic_packaging" : "no" }, { "glass_bottles" : "no", "clothes" : "yes" }, { "clothes" : "yes" }, { "glass" : "no", "glass_bottles" : "yes", "clothes" : "yes" }, { "glass_bottles" : "yes", "clothes" : "yes" }, { "glass_bottles" : "yes", "clothes" : "no" }, { "glass" : "yes" }, { "glass" : "yes", "clothes" : "no" }, { "glass_bottles" : "yes" }, { "glass" : "yes", "clothes" : "yes" } ], "count" : 32 }
{ "_id" : "centre", "type" : [ { "garden_waste" : "yes" } ], "count" : 2 }

# Conclusion

The dataset provides a lot of useful information in different directions (e. g. wineries, population, position information, recycling and so on). The dataset is cleaned a bit for this exercise. But further cleaning is obviously necessary. Also a checking of the data quality and entering additional information (e. g. about wineries) is worth it t provide additionally to OpenStreetMap.org.
The amount of user entries is not that high. A connection with social media (e. g. facebook) or wine related media (e. g. vivino.de) provides an increase in user entriese because a a lot of events and locations are wine related and published via these media. Also the quality about wine related information might be  improved by using other media s input.

# Appendix

## Statistical Overview without MongoDB Queries

0.1 Size of original file
0.2 Size of json-file
1. Number of nodes, ways and other data
2. Number of unique users 

In [6]:
# -*- coding: utf-8 -*-
# get original file size from exported file "Meddersheim_Bad-Muenster_Bad-Kreuznach_original"
# get number of data tags from exported file "Meddersheim_Bad-Muenster_Bad-Kreuznach_original"
# get number of unique users for this dataset
'''
Created on Dec 6, 2015

@author: anita
'''

import xml.etree.ElementTree as ET
import json
import pprint
import re
import os
from stat import *

street_map_file = "Meddersheim_Bad-Muenster_Bad-Kreuznach_original"
add_address = {}
CREATED = ["version", "changeset", "timestamp", "user", "uid"]
i = []

def read_file(street_map_file):
    json_data_array = []
    dic_tag = {}
    users = set()
    users = []
    file_orig = os.stat(street_map_file) #get statistical information from dataset
    file_orig_size = float(file_orig.st_size) #get file size
    for event, element in ET.iterparse(street_map_file):
        # get user IDs
        uid = element.get('uid')
        if uid !=None:
            users.append(uid)
        # count xml-tags
        if dic_tag.has_key(element.tag):
            dic_tag[element.tag] = dic_tag[element.tag] + 1
        else:
            dic_tag[element.tag] = 1
    users = set(users)
    file_json = os.stat("P3_Meddersheim_Germany") #get statistical information from dataset
    file_size = float(file_json.st_size) #get file size
    pprint.pprint("0.1 Size of original file from OSM [MB]:")
    pprint.pprint(file_orig_size/1000000) #get size of the original file in [MB]
    pprint.pprint("0.2 Size of json file from OSM [MB]:")
    pprint.pprint(file_size/1000000) #get size of the original file in [MB]
    pprint.pprint("1. Number of nodes, ways and other data:")
    pprint.pprint(dic_tag)
    pprint.pprint("2. Number if unique users:")
    pprint.pprint(len(users))
    return dic_tag, users

if __name__ == '__main__':
    read_file(street_map_file)

'0.1 Size of original file from OSM [MB]:'
50.786579
'0.2 Size of json file from OSM [MB]:'
72.502022
'1. Number of nodes, ways and other data:'
{'bounds': 1,
 'member': 24704,
 'meta': 1,
 'nd': 291833,
 'node': 218983,
 'note': 1,
 'osm': 1,
 'relation': 682,
 'tag': 94015,
 'way': 35578}
'2. Number if unique users:'
484
